In [2]:
import os
from openai import OpenAI
import pandas as pd
import numpy as np
import re
import string
import json
import csv
from tqdm import tqdm


In [ ]:
client = OpenAI(
    api_key="",  # Write down your api key here
)

In [11]:
SIGHAN_df = pd.read_csv("Wiki_data/Sighan_input_classifer_test.csv", encoding='utf-8')
Wiki_df = pd.read_csv("Wiki_data\inputClassifier_test.csv", encoding='utf-8')

In [59]:
input_prompt = """你是一個語音輸入指令分類器，請判斷使用者的語音是以下哪種類型：

0：一般語音內容（如敘述、聊天內容）

1：編輯指令（例如「請把某個字改成別的字」、「刪除某句話」）

2：操作指令（例如「停止錄音」、「送出目前內容」、「重新開始錄音」）

若屬於操作指令（2），請再細分為下列三種，並輸出對應的代號：

2-1：停止錄音指令
表示暫停錄音，不再接收語音辨識內容。
例句：停止錄音、錄到這邊就好

2-2：送出目前內容指令
表示確認、傳送目前的輸入內容。
例句：送出、傳出去

2-3：重新開始錄音（繼續說話）指令
表示使用者要「繼續錄音」，恢復語音辨識。常見於先前已停止錄音後，現在希望繼續。
例句：重新開始、繼續錄音

請根據輸入的句子，輸出下列代號其中之一：
0、1、2-1、2-2 或 2-3"""

In [12]:
predictions = []
ground_truths = []

In [15]:
for index, row in tqdm(SIGHAN_df.iterrows(), total=len(SIGHAN_df)):
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": row['text']}
            ]
        )
        # 擷取回應
        result = response.choices[0].message.content.strip()

        # 儲存預測與真實標籤
        predictions.append(result)
        ground_truths.append(row['label'])

    except Exception as e:
        print(f"Error at index {index}: {e}")
        predictions.append("error")
        ground_truths.append(row['label'])

100%|██████████| 474/474 [04:26<00:00,  1.78it/s]


In [16]:
new_samples = pd.DataFrame([
    {"text": "停止錄音", "label": "2-1"},  # 停止錄音指令
    {"text": "送出", "label": "2-2"},      # 送出內容指令
    {"text": "繼續錄音", "label": "2-3"}   # 重新開始指令
])

In [17]:
for index, row in tqdm(new_samples.iterrows(), total=len(new_samples)):
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": row['text']}
            ]
        )
        # 擷取回應
        result = response.choices[0].message.content.strip()

        # 儲存預測與真實標籤
        predictions.append(result)
        ground_truths.append(row['label'])

    except Exception as e:
        print(f"Error at index {index}: {e}")
        predictions.append("error")
        ground_truths.append(row['label'])

100%|██████████| 3/3 [00:01<00:00,  2.10it/s]


In [18]:
print("Predictions:", predictions)
print("Ground Truths:", ground_truths)

Predictions: ['0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '

In [30]:

from sklearn.metrics import f1_score, classification_report
# 移除無效預測
valid_preds = [p for p, gt in zip(predictions, ground_truths) if p != "error"]
valid_labels = [str(gt) for p, gt in zip(predictions, ground_truths) if p != "error"]

# 計算並輸出 F1-score 與分類報告
print("F1 score (macro):", f1_score(valid_labels, valid_preds, average='macro'))
print("Classification Report:\n", classification_report(valid_labels, valid_preds))

F1 score (macro): 0.9329105003523608
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       237
           1       1.00      1.00      1.00       237
         2-1       1.00      1.00      1.00         1
         2-2       1.00      1.00      1.00         1
         2-3       0.50      1.00      0.67         1

    accuracy                           1.00       477
   macro avg       0.90      1.00      0.93       477
weighted avg       1.00      1.00      1.00       477



In [23]:

SIGHAN_df = pd.concat([SIGHAN_df, new_samples], ignore_index=True)

output_df = pd.DataFrame({
    "text": SIGHAN_df["text"],
    "prediction": predictions,
    "label": ground_truths
})
output_df.to_csv("SIGHAN_predictions.csv", index=False, encoding="utf-8-sig")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

def merge_rare_classes(label):
    if label in ["2-1", "2-2", "2-3"]:
        return "2-x"
    return label

merged_labels = [merge_rare_classes(lbl) for lbl in valid_labels]
merged_preds = [merge_rare_classes(pred) for pred in valid_preds]

# 建立混淆矩陣
cm = confusion_matrix(merged_labels, merged_preds, labels=sorted(set(merged_labels + merged_preds)))

# 畫圖
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=sorted(set(merged_labels + merged_preds)),
            yticklabels=sorted(set(merged_labels + merged_preds)))
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.tight_layout()

# 儲存圖檔（PNG）
plt.savefig("SIGHAN_confusion_matrix.png", dpi=300)  # 你也可以改成 .pdf 或 .svg
plt.close()  # 避免在非互動環境多餘顯示


NameError: name 'valid_labels' is not defined

In [21]:
wiki_predictions = []
wiki_ground_truths = []

In [22]:
for index, row in tqdm(Wiki_df.iterrows(), total=len(Wiki_df)):
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": row['text']}
            ]
        )
        # 擷取回應
        result = response.choices[0].message.content.strip()

        # 儲存預測與真實標籤
        wiki_predictions.append(result)
        wiki_ground_truths.append(row['label'])

    except Exception as e:
        print(f"Error at index {index}: {e}")
        wiki_predictions.append("error")
        wiki_ground_truths.append(row['label'])

100%|██████████| 579/579 [05:04<00:00,  1.90it/s]


In [24]:
for index, row in tqdm(new_samples.iterrows(), total=len(new_samples)):
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": row['text']}
            ]
        )
        # 擷取回應
        result = response.choices[0].message.content.strip()

        # 儲存預測與真實標籤
        wiki_predictions.append(result)
        wiki_ground_truths.append(row['label'])

    except Exception as e:
        print(f"Error at index {index}: {e}")
        wiki_predictions.append("error")
        wiki_ground_truths.append(row['label'])

100%|██████████| 3/3 [00:01<00:00,  2.06it/s]


In [25]:
# 移除無效預測
valid_preds = [p for p, gt in zip(wiki_predictions, wiki_ground_truths) if p != "error"]
valid_labels = [str(gt) for p, gt in zip(wiki_predictions, wiki_ground_truths) if p != "error"]

# 計算並輸出 F1-score 與分類報告
print("F1 score (macro):", f1_score(valid_labels, valid_preds, average='macro'))
print("Classification Report:\n", classification_report(valid_labels, valid_preds))

F1 score (macro): 0.9892193827677698
Classification Report:
               precision    recall  f1-score   support

           0       0.94      1.00      0.97       225
           1       1.00      0.96      0.98       354
         2-1       1.00      1.00      1.00         1
         2-2       1.00      1.00      1.00         1
         2-3       1.00      1.00      1.00         1

    accuracy                           0.97       582
   macro avg       0.99      0.99      0.99       582
weighted avg       0.98      0.97      0.97       582



In [26]:
Wiki_df = pd.concat([Wiki_df, new_samples], ignore_index=True)

wiki_output_df = pd.DataFrame({
    "text": Wiki_df["text"],
    "prediction": wiki_predictions,
    "label": wiki_ground_truths
})
wiki_output_df.to_csv("Wiki_predictions.csv", index=False, encoding="utf-8-sig")


In [118]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

def merge_rare_classes(label):
    if label in ["2-1", "2-2", "2-3"]:
        return "2-x"
    return label

merged_labels = [merge_rare_classes(lbl) for lbl in valid_labels]
merged_preds = [merge_rare_classes(pred) for pred in valid_preds]

# 建立混淆矩陣
cm = confusion_matrix(merged_labels, merged_preds, labels=sorted(set(merged_labels + merged_preds)))

# 畫圖
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=sorted(set(merged_labels + merged_preds)),
            yticklabels=sorted(set(merged_labels + merged_preds)))
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.tight_layout()

# 儲存圖檔（PNG）
plt.savefig("Wiki_confusion_matrix.png", dpi=300)  # 你也可以改成 .pdf 或 .svg
plt.close()  # 避免在非互動環境多餘顯示


In [27]:
wiki_valid_preds = [p for p, gt in zip(wiki_predictions, wiki_ground_truths) if p != "error"]
wiki_valid_labels = [str(gt) for p, gt in zip(wiki_predictions, wiki_ground_truths) if p != "error"]


In [31]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from matplotlib.lines import Line2D

def merge_rare_classes(label):
    if label in ["2-1", "2-2", "2-3"]:
        return "2-x"
    return label

# 合併類別
merged_wiki_labels = [merge_rare_classes(lbl) for lbl in wiki_valid_labels]
merged_wiki_preds = [merge_rare_classes(pred) for pred in wiki_valid_preds]
labels_wiki_merged = sorted(set(merged_wiki_labels + merged_wiki_preds))
cm_wiki_merged = confusion_matrix(merged_wiki_labels, merged_wiki_preds, labels=labels_wiki_merged)

merged_labels = [merge_rare_classes(lbl) for lbl in valid_labels]
merged_preds = [merge_rare_classes(pred) for pred in valid_preds]
labels_merged = sorted(set(merged_labels + merged_preds))
cm_merged = confusion_matrix(merged_labels, merged_preds, labels=labels_merged)

# 放大字體整體設定
sns.set(font_scale=1.4)

# 畫圖：兩張子圖
fig, axes = plt.subplots(1, 2, figsize=(16, 6), gridspec_kw={'wspace': 0.3})

# 左圖：SIGHAN
sns.heatmap(cm_merged, annot=True, fmt="d", cmap="Blues",
            xticklabels=labels_merged, yticklabels=labels_merged,
            ax=axes[0], annot_kws={"size": 12})  # 格內字體大小
axes[0].set_title("SIGHAN Confusion Matrix", fontsize=16)
axes[0].set_xlabel("Predicted Label", fontsize=14)
axes[0].set_ylabel("True Label", fontsize=14)
axes[0].tick_params(labelsize=12)

# 右圖：Wiki
sns.heatmap(cm_wiki_merged, annot=True, fmt="d", cmap="Blues",
            xticklabels=labels_wiki_merged, yticklabels=labels_wiki_merged,
            ax=axes[1], annot_kws={"size": 12})
axes[1].set_title("Wiki Confusion Matrix", fontsize=16)
axes[1].set_xlabel("Predicted Label", fontsize=14)
axes[1].set_ylabel("True Label", fontsize=14)
axes[1].tick_params(labelsize=12)

# 中間虛線
line = Line2D([0.5, 0.5], [0.1, 0.9], transform=fig.transFigure,
              color='gray', linestyle='--', linewidth=1)
fig.add_artist(line)

# 儲存圖片
plt.savefig("SIGHAN_confusion_matrix_combined_with_line.png", dpi=300)
plt.close()


In [4]:
SIGHAN_labeler_df = pd.read_csv("Wiki_data\Sighan_modify_area_test.csv", encoding='utf-8')
Wiki_labeler_df = pd.read_csv("Wiki_data\modifyClassifier_test.csv", encoding='utf-8')

In [5]:
from transformers import BertTokenizer, BertModel
import ast
import Levenshtein

tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-multilingual-cased')

C:\Users\jack0\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\jack0\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
SIGHAN_labeler = pd.DataFrame(columns=['text', 'command','corrected_text'])

In [ ]:
import ast
from tqdm import tqdm

for index, row in tqdm(SIGHAN_labeler_df.iterrows(), total=len(SIGHAN_labeler_df)):
    # 將 text 用 [SEP] 分割成兩部分
    parts = row['text'].split('[SEP]')
    if len(parts) != 2:
        print(f"格式錯誤，跳過 index {index}")
        continue

    original_text = parts[0].strip()
    command_text = parts[1].strip()

    # 將原句 tokenize
    tokens = tokenizer.tokenize(row['text'])

    # 解析標籤
    labels = ast.literal_eval(row['label'])

    # 判斷指令中是否包含三種關鍵字
    if '刪除' in command_text:
        operation = 'delete'
    elif '新增' in command_text:
        operation = 'insert'
    elif '改成' in command_text:
        operation = 'replace'
    else:
        operation = None

    if operation is None:
        print(f"index {index} 未包含有效指令，跳過")
        SIGHAN_labeler = pd.concat([SIGHAN_labeler, pd.DataFrame({
        'text': [original_text],
        'command': [command_text],
    })], ignore_index=True)
        continue

    # 先找出標籤中的關鍵位置
    modify_indices = [i for i, lab in enumerate(labels) if lab == "B-Modify"]
    filling_indices = [i for i, lab in enumerate(labels) if lab == "B-Filling"]

    if operation == 'replace':
        # 假設你要把 B-Modify 位置的 token 換成 B-Filling 位置的 token
        if modify_indices and filling_indices:
            for m_i, f_i in zip(modify_indices, filling_indices):
                tokens[m_i] = tokens[f_i]

    elif operation == 'delete':
        # 刪除 B-Modify 標記位置的 token
        for idx in sorted(modify_indices, reverse=True):
            tokens.pop(idx)

    elif operation == 'insert':
        # 加入 B-Filling 標記位置的 token（這邊示範在 B-Filling 位置插入該 token，實際視情況調整）
        # 這邊範例是直接把 B-Filling 的 token插入到該位置
        for m_i, f_i in zip(modify_indices, filling_indices):
            tokens.insert(m_i, tokens[f_i])
    
    if '[SEP]' in tokens:
        tokens = tokens[:tokens.index('[SEP]')]

    corrected_text = tokenizer.convert_tokens_to_string(tokens)
    corrected_text = ''.join(tokens).replace('##', '')

    SIGHAN_labeler = pd.concat([SIGHAN_labeler, pd.DataFrame({
        'text': [original_text],
        'command': [command_text],
        'corrected_text': [corrected_text]
    })], ignore_index=True)

    print(f"原句: {original_text}")
    print(f"指令: {command_text}")
    print(f"操作: {operation}")
    print(f"結果: {corrected_text}")
    print("="*30)

In [ ]:
print("SIGHAN_labeler DataFrame:")
print(SIGHAN_labeler)

SIGHAN_labeler.to_csv("Wiki_data/SIGHAN_labeler_output.csv", index=False, encoding='utf-8')

In [29]:
df = pd.read_csv("Wiki_data/SIGHAN_labeler_output.csv", encoding='utf-8')

df_predictions = []
df_ground_truths = []

In [41]:
df_predictions_4o = []

In [12]:
prompt = """你是一個語音辨識錯誤修正助手，任務是根據使用者的語音指令來修改句子。  
使用者會提供一段「原始句子」與一個「修正指令」。  
「修正指令」會指出「原始句子」中的錯誤內容，並提供如何修正該錯誤的方式。

錯誤的描述方式可能會用以下這幾種形式表達（例如使用者以語音輸入時）：
1. 部首結構：如「人部的他」、「女部的她」  
2. 可拆解為兩個部件的字型：如「弓長張」  
3. 常見詞語搭配說明：如「辛苦的辛」、「知曉的曉」、「谷欠欲」

請根據指令對句子進行適當的新增、刪除或替換，並回傳**修改後的完整句子**。  
請勿解釋或輸出中間步驟。

範例：
原始句子：我明天會去新北市拜訪朋友  
修正指令：請把新北市改成台中市  
回覆：我明天會去台中市拜訪朋友

原始句子：我想要一杯咖啡  
修正指令：請在咖啡前加上冰的  
回覆：我想要一杯冰的咖啡

原始句子：這真的是非常非常非常重要的事情  
修正指令：請刪掉多餘的非常  
回覆：這真的是非常重要的事情

原始句子：真不得，下張項目可能為何？  
修正指令：請在得到的得前面新增知曉的曉  
回覆：真不曉得，下張項目可能為何？

原始句子：心中並存這早熟的人脈與稚的自我表現與不時會表現出叛逆的態度  
修正指令：把與其的與改成谷欠欲  
回覆：心中並存這早熟的人脈與幼稚的自我表現欲不時會表現出叛逆的態度

原始句子：他太衝動了，做事總是不經思考  
修正指令：請把人部的他改成女部的她  
回覆：她太衝動了，做事總是不經思考"""

In [42]:
for index, row in tqdm(df.iterrows(), total=len(df)):
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": f"原始句子：{row['text']}\n語音指令：{row['command']}"}
            ]
        )
        # 擷取回應
        result = response.choices[0].message.content.strip()

        # 儲存預測與真實標籤
        df_predictions_4o.append(result)
        #df_ground_truths.append(row['corrected_text'])

    except Exception as e:
        print(f"Error at index {index}: {e}")
        df_predictions_4o.append("error")
        #df_ground_truths.append(row['corrected_text'])

100%|██████████| 244/244 [04:35<00:00,  1.13s/it]


In [43]:
print("Predictions:", df_predictions_4o)
print(len(df_predictions_4o))
#df_ground_truths = df['corrected_text'].tolist()
#df['ground_truth'] = df_ground_truths
print("Ground Truths:", df_ground_truths)
print(len(df_ground_truths))

Predictions: ['你說的是對，跟那些失業的人比起來你也算是運的。', '雖然是有＂經濟＂，可是又影響到生態環境，導致台灣持有的動物可能因此消失，接下來旅客為此而不再來會導致台灣旅遊業損失。', '怎麼辦？其實阿偉一整天水樂不曲學校了！', '假如哪天你是被丟書包的人呢你會怎麼辦呢，你會他一樣這麼做嗎，還是有其他辦法呢。', '根據人生的自然來講，每個人都愛自己的家人，尤其是「兒子與媽媽的關係」，大家都會知道他們的愛是無法計算的。', '對不起老師，今天早上我的婆婆生病，所以我要帶她去看醫生。', '在上班的時候會專心地工作，回家後也會付出了很多愛心物。', '裝了攝影機以後，家長就不用為再一次的子女苦惱思想了。', '我覺得這些人也在第二個相片兒，也在第三個相片兒。這個男人也在第一個相片兒。這兩人是朋友。', '我真的很高興你已經找到了一份工作了。', '事實上我們社會的的確壞人很多。', '孩子在也要上高中了吧，你還可以輔導他的學習。', '我的家是在您的工廠附近，最近您的工廠目的的噪音和臭味都讓這裡的人民真受不了。', '不要因為那位同學覺得你是小偷而導致你對他有偏見，我相信他心一定比妳更尷尬更不舒服。', '後來我回想以下就有重幽靈一般的問候出現：「值日生」是什麼？', '在夜店很熱鬧，有很多辣妹在跳舞，還可以喝酒和啤酒，跳舞，跟辣妹聊一聊。', '我想問你明天請假，因為今天下課以後我發現我的車子被偷走了。', '你回美國已後，請你幫我買一個好看黑色的錶。', '如果不能力的改的話，我們去投訴政府的機關。', '她告訴愛文：如果他在晚上喝酒他的身體一定不太好。', '最近廠長裡面發出一種很難聞的味道，這種味道只是讓我們村裡的空氣汙染，而還讓大家覺得害怕。', '雖然我理解你的想法，但我個人已經有了自己的人生計畫。', '今天他學習得很愉快。', '問題不在出來外面、問題一定是在個人裡面。', '雖然服用藥物我們還在呼吸外面的空氣、也還在接處贓的東西、怎麼可以容易治療呢？', '有人提起「隱私權」的問題，當然了解他們的想法，但跟小孩的學校生活安全比起來哪一個是重要呢？', '花蓮台東的風景氣勢磅礡、澎湖的景觀是人間仙境，要是這兩個地方的可貴自然美都被破壞了，台灣反而就是去了觀光的吸引力。', '他到學校的時時，中文課已經開始。', '問題就是「半工半

In [44]:
df['prediction_4o_mini'] = df_predictions_4o
#df['ground_truth'] = df_ground_truths
#df['ground_truth'] = SIGHAN_labeler['corrected_text']
df.to_csv("SIGHAN_LLM_predictions.csv", index=False)

In [45]:
df['is_correct'] = df['prediction_4o_mini'] == df['ground_truth']
accuracy = df['is_correct'].mean()
print(f"Accuracy: {accuracy:.2%}")

Accuracy: 48.77%


In [92]:
df.to_csv("SIGHAN_LLM_predictions.csv", index=False)

In [9]:
Wiki_labeler = pd.DataFrame(columns=['text', 'command','corrected_text'])

----

In [ ]:

for index, row in tqdm(Wiki_labeler_df.iterrows(), total=len(Wiki_labeler_df)):
    # 將 text 用 [SEP] 分割成兩部分
    parts = row['text'].split('[SEP]')
    if len(parts) != 2:
        print(f"格式錯誤，跳過 index {index}")
        continue

    original_text = parts[0].strip()
    command_text = parts[1].strip()

    # 將原句 tokenize
    tokens = tokenizer.tokenize(row['text'])

    # 解析標籤
    labels = ast.literal_eval(row['label'])

    # 判斷指令中是否包含三種關鍵字
    if '刪除' in command_text:
        operation = 'delete'
    elif '新增' in command_text:
        operation = 'insert'
    elif '改成' in command_text:
        operation = 'replace'
    else:
        operation = None

    if operation is None:
        print(f"index {index} 未包含有效指令，跳過")
        Wiki_labeler = pd.concat([Wiki_labeler, pd.DataFrame({
        'text': [original_text],
        'command': [command_text],
    })], ignore_index=True)
        continue

    # 先找出標籤中的關鍵位置
    modify_indices = [i for i, lab in enumerate(labels) if lab == "B-Modify"]
    filling_indices = [i for i, lab in enumerate(labels) if lab == "B-Filling"]

    if operation == 'replace':
        # 假設你要把 B-Modify 位置的 token 換成 B-Filling 位置的 token
        if modify_indices and filling_indices:
            for m_i, f_i in zip(modify_indices, filling_indices):
                tokens[m_i] = tokens[f_i]

    elif operation == 'delete':
        # 刪除 B-Modify 標記位置的 token
        for idx in sorted(modify_indices, reverse=True):
            tokens.pop(idx)

    elif operation == 'insert':
        # 加入 B-Filling 標記位置的 token（這邊示範在 B-Filling 位置插入該 token，實際視情況調整）
        # 這邊範例是直接把 B-Filling 的 token插入到該位置
        for m_i, f_i in zip(modify_indices, filling_indices):
            tokens.insert(m_i, tokens[f_i])
    
    if '[SEP]' in tokens:
        tokens = tokens[:tokens.index('[SEP]')]

    corrected_text = tokenizer.convert_tokens_to_string(tokens)
    corrected_text = ''.join(tokens).replace('##', '')

    Wiki_labeler = pd.concat([Wiki_labeler, pd.DataFrame({
        'text': [original_text],
        'command': [command_text],
        'corrected_text': [corrected_text]
    })], ignore_index=True)

    print(f"原句: {original_text}")
    print(f"指令: {command_text}")
    print(f"操作: {operation}")
    print(f"結果: {corrected_text}")
    print("="*30)

In [11]:
print("Wiki_labeler DataFrame:")
print(Wiki_labeler)
Wiki_labeler.to_csv("Wiki_data/Wiki_labeler_output.csv", index=False, encoding='utf-8')

Wiki_labeler DataFrame:
                                                  text        command  \
0    該域名與 1995 年 4 月 29 日啟用不知別克斯坦和拉脫維亞的一些網站使用的是該域名該...    把不懈的不改成烏龜的烏   
1    1234 四本紀奈是一種多環芳香烴常用於本科生的實驗室教學中的迪爾斯阿爾德反應教學在這個例子...      把彳亍行改成刑土型   
2                          是沒現於東青塘燈旗魚從廣運通將當更正射頻想去參以做運維      把厽㐱參改成殸耳聲   
3                            急所謂普選時間表明定哪一年能補選政府建議中沒有提及     把刍心急改成旋即的即   
4    台東市第三公有零售市場與 1997 年新建會在台東市仁二街 310 巷內惟台東縣政府之公共造...         刪除會計的會   
..                                                 ...            ...   
496       1862 年第二次那沙司之一爆發維吉尼亞軍團得勝而受鼓舞主動向北方發動侵略展開馬裡蘭會戰  在那兒的那前面新增跑馬的馬   
497  糖心線高架路舊村三省道東阜縣高價是浙江省杭州市的一條高架快速路起點位於紅 15 線終點目前位...      把𠂤十阜改成衤复複   
498  事故發生後初期臺灣輿論認為只是尋常的大陸赴台遊客交通意外有此大規模討論遊覽車的安全問題先揭露...      把水步涉改成言殳設   
499  該域名與 1995 年 1 月 25 日期用的虛是尼泊爾和在尼泊爾有辦事機構的實體組織和公司...     把禾口和改成間或的或   
500  具景觀回憶描述左右鄰居都跟他吵過或被他告過而且隨身會帶一台相機蒐證檢舉他心機很重難以相處的個...    在粦阝鄰前面新增土也地   

                                        corrected_text  
0    該域名與1995年4月29日啟用烏知別克斯坦和拉脫維亞的一些網站使用的是該

In [18]:
Wiki_output = []

In [ ]:
Wiki_output_4o = []

In [24]:
Wiki_output_4o_mini = []

In [25]:
for index, row in tqdm(Wiki_labeler.iterrows(), total=len(Wiki_labeler)):
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": f"原始句子：{row['text']}\n修正指令：{row['command']}"}
            ]
        )
        # 擷取回應
        result = response.choices[0].message.content.strip()

        # 儲存預測與真實標籤
        Wiki_output_4o_mini.append(result)
        #df_ground_truths.append(row['corrected_text'])

    except Exception as e:
        print(f"Error at index {index}: {e}")
        Wiki_output_4o_mini.append("error")
        #df_ground_truths.append(row['corrected_text'])

100%|██████████| 501/501 [13:39<00:00,  1.64s/it]


In [26]:
Wiki_labeler['prediction_4o_mini'] = Wiki_output_4o_mini
#Wiki_labeler.to_csv("Wiki_LLM_predictions.csv", index=False)

In [27]:
Wiki_labeler['prediction_4o_mini'] = Wiki_labeler['prediction_4o_mini'].str.replace(' ', '', regex=False)
print(Wiki_labeler['prediction_4o_mini'])

0      該域名與1995年4月29日啟用烏龜知別克斯坦和拉脫維亞的一些網站使用的是該域名該網站沒有註...
1      原始句子中並沒有「彳亍行」，而是出現了「環」字，因此無法執行修正指令。如果有其他需要修改的內...
2                           是沒現於東青塘燈旗魚從廣運通將當更正射頻想去殸耳聲做運維
3                              急所謂普選時間表明定哪一年能補選政府建議中沒有提及
4      台東市第三公有零售市場與1997年新建在台東市仁二街310巷內惟台東縣政府之公共造產遊台東市...
                             ...                        
496         1862年第二次馬那沙司之一爆發維吉尼亞軍團得勝而受鼓舞主動向北方發動侵略展開馬裡蘭會戰
497    糖心線高架路舊村三省道東複縣高價是浙江省杭州市的一條高架快速路起點位於紅15線終點目前位於東...
498    事故發生後初期臺灣輿論認為只是尋常的大陸赴台遊客交通意外有此大規模討論遊覽車的安全問題先揭露...
499    該域名與1995年1月25日期用的虛是尼泊爾間或在尼泊爾有辦事機構的實體組織和公司才能申請註冊域名
500    具景觀回憶描述左右土也地鄰居都跟他吵過或被他告過而且隨身會帶一台相機蒐證檢舉他心機很重難以相...
Name: prediction_4o_mini, Length: 501, dtype: object


In [28]:
Wiki_labeler['is_correct_4o_mini'] = Wiki_labeler['prediction_4o_mini'] == Wiki_labeler['corrected_text']
accuracy = Wiki_labeler['is_correct_4o_mini'].mean()
print(f"Accuracy: {accuracy:.2%}")

Accuracy: 14.57%


In [32]:
Wiki_labeler.to_csv("Wiki_LLM_predictions.csv", index=False)

In [ ]:
print("Wiki_labeler DataFrame:")
print(Wiki_labeler)

In [46]:
annotator_df = pd.read_csv("llm_testing.csv", encoding='utf-8')

In [47]:
annotator_predictions = []

In [48]:
for index, row in tqdm(annotator_df.iterrows(), total=len(annotator_df)):
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": f"原始句子：{row['ASR_Result']}\n語音指令：{row['command']}"}
            ]
        )
        # 擷取回應
        result = response.choices[0].message.content.strip()

        # 儲存預測與真實標籤
        annotator_predictions.append(result)
        #df_ground_truths.append(row['corrected_text'])

    except Exception as e:
        print(f"Error at index {index}: {e}")
        annotator_predictions.append("error")
        #df_ground_truths.append(row['corrected_text'])

100%|██████████| 7/7 [00:10<00:00,  1.54s/it]


In [49]:
annotator_df['prediction_4o_mini'] = annotator_predictions

In [50]:
annotator_predictions_4o = []
for index, row in tqdm(annotator_df.iterrows(), total=len(annotator_df)):
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": f"原始句子：{row['ASR_Result']}\n語音指令：{row['command']}"}
            ]
        )
        # 擷取回應
        result = response.choices[0].message.content.strip()

        # 儲存預測與真實標籤
        annotator_predictions_4o.append(result)
        #df_ground_truths.append(row['corrected_text'])

    except Exception as e:
        print(f"Error at index {index}: {e}")
        annotator_predictions_4o.append("error")
        #df_ground_truths.append(row['corrected_text'])

100%|██████████| 7/7 [00:09<00:00,  1.36s/it]


In [51]:
annotator_df['prediction_4o'] = annotator_predictions_4o

In [52]:
annotator_df['is_correct_4o'] = annotator_df['prediction_4o'] == annotator_df['corrected_text']
accuracy = annotator_df['is_correct_4o'].mean()
print(f"Accuracy: {accuracy:.2%}")

Accuracy: 57.14%


In [53]:
annotator_df['is_correct_4o_mini'] = annotator_df['prediction_4o_mini'] == annotator_df['corrected_text']
accuracy = annotator_df['is_correct_4o_mini'].mean()
print(f"Accuracy: {accuracy:.2%}")

Accuracy: 71.43%


In [54]:
annotator_df.to_csv("annotator_LLM_predictions_v2.csv", index=False)

### Speed Test

In [56]:
wiki_df = pd.read_csv("Wiki_data\Wiki_labeler_output.csv", encoding='utf-8')
sighan_df = pd.read_csv("Wiki_data\Sighan_labeler_output.csv", encoding='utf-8')

In [57]:
wiki_sample = wiki_df.sample(n=10, random_state=42)  # 設定 random_state 以確保可重現
sighan_sample = sighan_df.sample(n=10, random_state=42)

# 合併 sample
combined_sample = pd.concat([wiki_sample, sighan_sample], ignore_index=True)

# 輸出成 CSV
combined_sample.to_csv("sampled_20_examples.csv", index=False, encoding='utf-8')

In [ ]:
for index, row in tqdm(combined_sample.iterrows(), total=len(combined_sample)):

    try:
        # GPT-4o-mini 預測
        response_mini = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": input_prompt},
                {"role": "user", "content": row['command']}
            ]
        )
        result_mini = response_mini.choices[0].message.content.strip()
        print(f"[{index}] GPT-4o-mini:\n{result_mini}\n")

    except Exception as e:
        print(f"[GPT-4o-mini] Error at index {index}: {e}")

    try:
        # GPT-4o 預測
        response_full = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": f"原始句子：{row['text']}\n修正指令：{row['command']}"}
            ]
        )
        result_full = response_full.choices[0].message.content.strip()
        print(f"[{index}] GPT-4o:\n{result_full}\n")

    except Exception as e:
        print(f"[GPT-4o] Error at index {index}: {e}")